In [1]:
import pandas as pd

academy_awards = pd.read_csv('academy_awards.csv', encoding = 'ISO-8859-1')
academy_awards.head(5)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


We'll work with data on Academy Award nominations. The Academy Awards, also known as the Oscars, is an annual awards ceremony hosted to recognize the achievements in the film industry. There are many different awards categories and the members of the academy vote every year to decide which artist or film should get the award. The awards categories have changed over the years, and you can learn more about when categories were added on Wikipedia.

Here are the columns in the dataset, academy_awards.csv:

Year - the year of the awards ceremony.
Category - the category of award the nominee was nominated for.
Nominee - the person nominated for the award.
Additional Info - this column contains additional info like:
the movie the nominee participated in.
the character the nominee played (for acting awards).
Won? - this column contains either YES or NO depending on if the nominee won the award.

In [2]:
academy_awards['Year'] = academy_awards['Year'].str[0:4]
academy_awards.head(5)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


The dataset is incredibly messy and we have noticed many inconsistencies that make it hard to work with. Most columns don't have consistent formatting, which is incredibly important when we use SQL to query the data later on. Other columns vary in the information they convey based on the type of awards category that row corresponds to.

This subset contained only the nominations from years 2001 to 2010 and only the following awards categories:

Actor -- Leading Role
Actor -- Supporting Role
Actress -- Leading Role
Actress -- Supporting Role
Let's filter our Dataframe to the same subset so it's more manageable.

In [3]:
academy_awards['Year'] = academy_awards['Year'].astype(dtype='int64')

In [4]:
later_than_2000 = academy_awards[academy_awards['Year'] > 2000]

In [5]:
award_categories = ['Actor -- Leading Role','Actor -- Supporting Role',
           'Actress -- Leading Role','Actress -- Supporting Role']
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

Since SQLite uses the integers 0 and 1 to represent Boolean values, convert the Won? column to reflect this. Also rename the Won? column to Won so that it's consistent with the other column names. Finally, get rid of the 6 extra, unnamed columns, since they contain only null values in our filtered Dataframe nominations.

In [7]:
replace_dict = { "YES": 1, "NO": 0 }

In [8]:
nominations['Won?'] = nominations['Won?'].map(replace_dict)

/opt/dsserver/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
nominations.head(5)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
nominations['Won'] = nominations['Won?']

/opt/dsserver/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
final_nominations = nominations.drop(['Won?','Unnamed: 5','Unnamed: 6',
                                     'Unnamed: 7','Unnamed: 8',
                                     'Unnamed: 9','Unnamed: 10'],1)

Now let's clean up the Additional Info column, whose values are formatted like so:

MOVIE {'CHARACTER'}

Here are some examples:

Biutiful {'Uxbal'} - Biutiful is the movie and Uxbal is the character this nominee played.
True Grit {'Rooster Cogburn'} - True Grit is the movie and Rooster Cogburn is the character this nominee played.
The Social Network {'Mark Zuckerberg'} - The Social Network is the movie and Mark Zuckerberg is the character this nominee played.
The values in this column contain the movie name and the character the nominee played. Instead of keeping these values in 1 column, split them up into 2 different columns for easier querying.

In [12]:
additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")

In [15]:
additional_info_two = additional_info_one.str.split("{'")

In [16]:
additional_info_two.head(5)

0                        [Biutiful , Uxbal]
1             [True Grit , Rooster Cogburn]
2    [The Social Network , Mark Zuckerberg]
3      [The King's Speech , King George VI]
4                [127 Hours , Aron Ralston]
Name: Additional Info, dtype: object

In [17]:
movie_names = additional_info_two.str[0]

In [18]:
final_nominations['Movie'] = movie_names

In [19]:
Characters = additional_info_two.str[1]

In [20]:
final_nominations['Character'] = Characters

In [21]:
final_nominations.head(5)

,Year,Category,Nominee,Additional Info,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,127 Hours,Aron Ralston


In [22]:
import sqlite3

conn = sqlite3.connect("nominations.db")

In [23]:
final_nominations.to_sql("nominations",conn,index=False)

/opt/dsserver/lib/python3.4/site-packages/pandas/io/sql.py:1309: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  warnings.warn(_SAFE_NAMES_WARNING)


In [25]:
cursor = conn.cursor()

In [28]:
pragma table_info("nominations");

SyntaxError: invalid syntax (<ipython-input-28-bff46d92633b>, line 1)

In [29]:
cursor.execute("Select * from nominations limit 5;").fetchall()

[(2010,
  'Actor -- Leading Role',
  'Javier Bardem',
  "Biutiful {'Uxbal'}",
  0,
  'Biutiful ',
  'Uxbal'),
 (2010,
  'Actor -- Leading Role',
  'Jeff Bridges',
  "True Grit {'Rooster Cogburn'}",
  0,
  'True Grit ',
  'Rooster Cogburn'),
 (2010,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  "The Social Network {'Mark Zuckerberg'}",
  0,
  'The Social Network ',
  'Mark Zuckerberg'),
 (2010,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech {'King George VI'}",
  1,
  "The King's Speech ",
  'King George VI'),
 (2010,
  'Actor -- Leading Role',
  'James Franco',
  "127 Hours {'Aron Ralston'}",
  0,
  '127 Hours ',
  'Aron Ralston')]

In [30]:
conn.close()